## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Описание данных:
*  children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

### Поставлена задача ответить на вопросы:
* Есть ли зависимость между наличием детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

### План выполнения работы

1. Откройте таблицу и изучите общую информацию о данных
2. Предобработка данных
   * определите и заполните пропущенные значения:
   * опишите, какие пропущенные значения вы обнаружили;
   * приведите возможные причины появления пропусков в данных;
   * объясните, по какому принципу заполнены пропуски;
   * замените вещественный тип данных на целочисленный:
   * поясните, как выбирали метод для изменения типа данных;
   * удалите дубликаты:
   * поясните, как выбирали метод для поиска и удаления дубликатов в данных;
   * приведите возможные причины появления дубликатов;
   * выделите леммы в значениях столбца с целями получения кредита:
   * опишите, как вы проводили лемматизацию целей кредита;
   * категоризируйте данные
3. Ответьте на вопросы
   * Есть ли зависимость между наличием детей и возвратом кредита в срок?
   * Есть ли зависимость между семейным положением и возвратом кредита в срок?
   * Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
   * Как разные цели кредита влияют на его возврат в срок?
4. Напишите общий вывод


### Шаг 1. Откройте файл с данными и изучите общую информацию. 

Загружаем необходимые для выполнения задания библиотеки

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

читаем файл csv и присваеваем его переменной borrower_base (база заёмщиков)

In [2]:
borrower_base = pd.read_csv('/datasets/data.csv')

выводим общую информацию о dataframe с помощью метода df.info() и df.describe()

In [3]:
borrower_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
borrower_base.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Судя по количеству записей в столбцах, два из них имеют пропуски:
1. days_employed
2. total_income

но мы дополнительно проверим это позднее с помощью методов isnull/insna

столбцы:
1. children 
2. days_emploeyd

имеют отрицательные значения, что невохможно



Выводим первые 10 строк таблицы, чтобы поверхностно ознакомиться с данными, которые мы будем обрабатывать далеее.

In [5]:
display(borrower_base.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

После первичного осмотра выявлены следующие проблемы:
1. в столбце days_employed есть отрицательные значения ( такого быть не может, так как трудовой стаж не может быть отрицательным)
2. в столбце education есть одинаковые названия, необходимо обработать столбец для дальнейшего анализа.  
3. Могу предположить, что пропуски возникли по следуюзим причинам:
    * ошибки ручного ввода оператором
    * заёмщики, информация о которых расположенна в строках с пропусками, не работают, поэтому нет стажа и дохода


### Шаг 2. Предобработка данных

### Обработка пропусков

Используя метод insull() находим пустые ячейки в таблице

In [6]:
display(borrower_base.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Используя метод isna() находим пустые ячейки в таблице

In [7]:
display(borrower_base.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Выводим на экран строки с пропусками в столбце days_employed


In [8]:
display(borrower_base[borrower_base['days_employed'].isnull()])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Мы видим,что строки где есть пропуски в столбце days_employed имеют пропуски и в столбце total_income.

удалять строки с пропусками целиком мы не можем, так мы потеряем важную для нас информацию в других столбцах.
а 2174 строки это 10% данных, что очень значительная потеря

1. так как данные в столбце days_employed никак не помогают нам в ответе на поставленные перед нами вопросы, мы можем не заниматься дальнейшей их обработкой и заменить все пропуски на нули для дальнейшенго удобства.

2. столбец total_income важен для нас в дальнейших расчётах так как сумма дохода помогает нам ответить на один из поставленных нам вопросов.
    1. Мы можем запросить у разработчиков уточнить данные по пропускам в столбце total income
    2. Мы можем поставить отрицательное значение вместо пропусков в столбце total income, так как оно не будет нам мешать в дальнейших расчётах, потому что отрицательного дохода быть не может и это не попадает не под одну категорию, которую мы будем использовать дальше. 
    
    3. Заполнить пропуски по среднему значению для группы.

я выбираю второй вариант действий со столбцом total income

заменяем все пропуски в столбце days_employed на 0.

In [9]:
borrower_base['days_employed'] = borrower_base['days_employed'].fillna(0)

проверяем замену

In [10]:
borrower_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


заменяем все пропуски в столбце total_income на отрицательное значение -1

In [11]:
borrower_base['total_income'] = borrower_base['total_income'].fillna(-1)

проверяем замену

In [12]:
borrower_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Обнаружены пропуски в двух столбцах таблицы.
один из них не критичен для дальнейших расчётов (days_employed), его пропуски мы просто заполнили нулями.
Другой столбец критичен для дальшейшего исследования, его пропуски мы заполнили отрицательными значения, чтобы в дальшейм они не портили выводы из результатов исследования

### Замена типа данных

С помощью метода astype заменить типа данных у столбцов :days_employed и total_income.

In [13]:
borrower_base['days_employed'] = borrower_base['days_employed'].astype('int')
borrower_base['total_income'] = borrower_base['total_income'].astype('int')

проверяем результат смены типа данных

In [14]:
borrower_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Был выбран метод astype так как он может переводить значения в формат int. Согласно заданию "замените вещественный тип данных на целочисленный" нам необходим именно этот типа данных.

метод to_nomeric() переводит во float, поэтому его использовать нельзя.
Другие методы в курсе не были показаны ( или я их не увидел =) )

### Обработка дубликатов

посчитаем количество дубликатов

In [15]:
print('Количество дубликатов',borrower_base.duplicated().sum())

Количество дубликатов 54


54 это очень маленькое количество дубликатов. всего 0.002% от таблицы. 
Поэтому я считаю, что могу удалить их не повлияя значительно на результат исследований


In [16]:
borrower_base = borrower_base.drop_duplicates().reset_index(drop = True)

проверяем удаление дубликатов

In [17]:
print('Количество дубликатов',borrower_base.duplicated().sum())

Количество дубликатов 0


### Вывод

Количество дубликатов не критично для общего итога исследования, поэтому было принято решение их просто удалить.

### Обработка отрицательных значений

Посмотрим общую статистику по таблице.

In [18]:
borrower_base.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21471.000000,21471.000000,21471.000000,21471.000000,21471.000000,21471.000000,2.147100e+04
mean,0.539565,56821.712496,43.279074,0.817195,0.973685,0.081086,1.508908e+05
std,1.382978,135009.925299,12.574291,0.548508,1.421082,0.272974,1.097755e+05
min,-1.000000,-18388.000000,0.000000,0.000000,0.000000,0.000000,-1.000000e+00
25%,0.000000,-2522.000000,33.000000,1.000000,0.000000,0.000000,8.894650e+04
50%,0.000000,-989.000000,42.000000,1.000000,0.000000,0.000000,1.357160e+05
75%,1.000000,0.000000,53.000000,1.000000,1.000000,0.000000,1.957515e+05
max,20.000000,401755.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


у нас есть отрицательные значения в столбца children и days_employed.
Так как days_employed для далнейших расчётаов нам не нужна, мы её трогать не будем.

посмотрим на отрицательные значения в столбце 'children'

In [19]:
display(borrower_base[borrower_base['children'] < 0])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,-4417,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816,профильное образование
705,-1,-902,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882,приобретение автомобиля
742,-1,-3174,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268,дополнительное образование
800,-1,349987,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293,дополнительное образование
941,-1,0,57,Среднее,1,женат / замужем,0,F,пенсионер,0,-1,на покупку своего автомобиля
1363,-1,-1195,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550,профильное образование
1929,-1,-1461,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121,покупка жилья
2073,-1,-2539,42,среднее,1,в разводе,3,F,компаньон,0,162638,покупка жилья
3813,-1,-3045,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892,на проведение свадьбы
4199,-1,-901,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375,операции со своей недвижимостью


посчитаем эти значения

In [20]:
display(borrower_base[borrower_base['children'] < 0]['children'].count())

47

посчитаем все строчки, больше или равные 0 

In [21]:
display(borrower_base[borrower_base['children'] >= 0]['children'].count())

21424

### Вывод

всего 47 значений из 21424.
В этом случае считаю, что эти значения можно принебречь.

отрицательные значения, могли быть вызваны неправильнйо выгрузкой исходных данных или ошибкой при вводе.
Можно уточнить их у разработчиков, но повторюсь в данном случае это меньше 1% ими можно принебречь.

### Лемматизация

Пишем функцию для лемматизации значений столбцов purpose.

In [22]:
def make_lemm(row):
    text = row
    lemmas = m.lemmatize(text)
    return lemmas

Создаем новый столбец с лемматизированными значениями.

In [23]:
borrower_base['purpose_lemms'] = borrower_base['purpose'].apply(make_lemm)

Проверяем результат.

In [24]:
display(borrower_base.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemms
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]"
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]"
7,0,-152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]"
8,2,-6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]"


лемматизация прошла успешно

Создаем список со всеми леммами.

In [25]:
purpose_list=[]
for i in range(0,21454):
    querys = borrower_base.loc[i,'purpose_lemms']
    for query in querys:
        purpose_list.append(query)


Выводим количесвто лемм.

In [26]:
print(Counter(purpose_list))

Counter({' ': 33567, '\n': 21454, 'недвижимость': 6348, 'покупка': 5893, 'жилье': 4457, 'автомобиль': 4302, 'образование': 4012, 'с': 2914, 'операция': 2602, 'свадьба': 2335, 'свой': 2228, 'на': 2225, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1231, 'сделка': 939, 'дополнительный': 906, 'заниматься': 903, 'проведение': 773, 'сыграть': 769, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 460, 'профильный': 436})


### Вывод

Можем выделить следующе категории самых популярных целей кредита:
1. недвижимость и жилье
2. образование
3. автомобиль
4. свадьба

### Категоризация данных

#### Категоризация целей кредита

пишем функцию для добавление категорий по "цели кредита"

In [27]:
def lemm_type(row):
    for word in row:
        if word =='жилье' or word =='недвижимость':
            return 'недвижимость'
        elif word =='свадьба':
            return 'свадьба'
        elif word == 'автомобиль':
            return 'автомобиль'
        elif word =='образование':
            return 'образование'
        


Добавляем новый столбец с целями.

In [28]:
borrower_base['lem_type'] = borrower_base['purpose_lemms'].apply(lemm_type)

проверяем добавленные категории

In [29]:
display(borrower_base['lem_type'].value_counts())

недвижимость    10814
автомобиль       4308
образование      4014
свадьба          2335
Name: lem_type, dtype: int64

#### Категоризация по количеству детей

Смотрим количество значений по этому параметру

In [30]:
display(borrower_base['children'].value_counts())

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Согласно градации в википедии многодетными считаются семьи имеющие от 3х детей.
Выделяем следующие категории:
1. многодетные
2. 1 ребёнок
3. 2 ребёнка
4. нет детей

отрицательные значения мы ранее решили не учитывать в данной классификации.

[Википедия](https://ru.wikipedia.org/wiki/%D0%9C%D0%BD%D0%BE%D0%B3%D0%BE%D0%B4%D0%B5%D1%82%D0%BD%D0%B0%D1%8F_%D1%81%D0%B5%D0%BC%D1%8C%D1%8F_%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8)

пишем функцию для категоризации по столбцу 'children'

In [31]:
def family_type(row):
    if row == 0:
        return 'нет детей'
    elif   row == 2:
        return '2 ребёнка'
    elif row== 1:
        return '1 ребёнок'
    else:
        return 'многодетные'

применияем фукнцию

In [32]:
borrower_base['family_type'] = borrower_base['children'].apply(family_type)

проверяем работу функции

In [33]:
display(borrower_base.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemms,lem_type,family_type
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,1 ребёнок
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,1 ребёнок
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,нет детей
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,многодетные
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,нет детей


считаем количество по категориям.

In [34]:
display(borrower_base['family_type'].value_counts())

нет детей      14107
1 ребёнок       4809
2 ребёнка       2052
многодетные      503
Name: family_type, dtype: int64

#### Категоризация по доходу

пишем функцию для категоризации доходов заёмщиков.
для разработки категорий были использованы следующие данные [ссылка](https://www.bnkomi.ru/data/news/59037/)

[]()

In [35]:
def total_income_type(row):
    if 0<=row <90000:
        return 'ниже среднего'
    elif 90000<= row <130000:
        return 'средний достаток'
    elif 130000 <= row <170000:
        return 'выше среднего'
    elif 170000 <= row:
        return 'богатые'

добавляем новый столбец

In [36]:
borrower_base['total_income_type'] = borrower_base['total_income'].apply(total_income_type)

проверяем результаты

In [37]:
display(borrower_base.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemms,lem_type,family_type,total_income_type
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,1 ребёнок,богатые
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,1 ребёнок,средний достаток
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,нет детей,выше среднего
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,многодетные,богатые
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,нет детей,выше среднего
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",недвижимость,нет детей,богатые
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,нет детей,богатые
7,0,-152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование,нет детей,выше среднего
8,2,-6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,2 ребёнка,средний достаток
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость,нет детей,выше среднего


проверяем сортировку

In [38]:
display(borrower_base['total_income_type'].value_counts())

богатые             7216
средний достаток    4594
выше среднего       4193
ниже среднего       3348
Name: total_income_type, dtype: int64

### Вывод

все категоризации проведены, можем приступать к исследованию и ответу на заданные вопросы.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

создаем сводную таблицу 

In [39]:
borrower_base_pivot_children = borrower_base.pivot_table(index='family_type',
values = ['debt'], aggfunc = ['sum','count'])
borrower_base_pivot_children['percent'] = borrower_base_pivot_children['sum'] / borrower_base_pivot_children['count']
borrower_base_pivot_children = borrower_base_pivot_children.sort_values(by='percent')

смотрим результаты

In [40]:
display(borrower_base_pivot_children)


,sum,count,percent
,debt,debt,
family_type,,,
нет детей,1063,14107,0.075353
многодетные,40,503,0.079523
1 ребёнок,444,4809,0.092327
2 ребёнка,194,2052,0.094542


### Вывод

Кличество детей зависит на возврат кредита в срок.  
Меньше всего просрочек и бездетных и многодетных.

у людей с 1 или 2 детьми просрочки случаются чаще.

### - Есть ли зависимость между семейным положением и возвратом кредита в срок?

создаем сводную таблицу

In [41]:
borrower_base_pivot_family_status = borrower_base.pivot_table(index='family_status' ,
values = 'debt', aggfunc = ['sum','count'])
borrower_base_pivot_family_status['percent'] = borrower_base_pivot_family_status['sum'] / borrower_base_pivot_family_status['count']
borrower_base_pivot_family_status = borrower_base_pivot_family_status.sort_values(by='percent')

смотрим результаты

In [42]:
display(borrower_base_pivot_family_status)

,sum,count,percent
,debt,debt,
family_status,,,
вдовец / вдова,63,959,0.065693
в разводе,85,1195,0.071130
женат / замужем,931,12344,0.075421
гражданский брак,388,4163,0.093202
Не женат / не замужем,274,2810,0.097509


### Вывод

Есть завимосость между семейным положение и возвратом кредита в срок.  
Меньше всего просрочек у вдовцов и вдов, а больше всего у одиноких.
можно сказать, что брак помогает лучше планировать бюджет, а жизнь после брака пропорционально меньше тратить.



###  Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [43]:
borrower_base_pivot_income = borrower_base.pivot_table(index='total_income_type' ,
values = 'debt', aggfunc = ['sum','count'])
borrower_base_pivot_income['percent'] = borrower_base_pivot_income['sum'] / borrower_base_pivot_income['count']
borrower_base_pivot_income = borrower_base_pivot_income.sort_values(by='percent')


In [44]:
display(borrower_base_pivot_income)

,sum,count,percent
,debt,debt,
total_income_type,,,
богатые,547,7216,0.075804
ниже среднего,262,3348,0.078256
средний достаток,384,4594,0.083587
выше среднего,378,4193,0.090150


### Вывод

чаше всего просрачиват люди с достатком выше среднего.
Люди с достатком ниже среднего и богатые лучше рассчитывают свой бюджет.

### - Как разные цели кредита влияют на его возврат в срок?

создаем сводную таблицу

In [45]:
borrower_base_pivot_purpose = borrower_base.pivot_table(index='lem_type' ,
values = 'debt', aggfunc = ['sum','count'])
borrower_base_pivot_purpose['percent'] = borrower_base_pivot_purpose['sum'] / borrower_base_pivot_purpose['count']
borrower_base_pivot_purpose = borrower_base_pivot_purpose.sort_values(by='percent')

смотрим результат

In [46]:
display(borrower_base_pivot_purpose)

,sum,count,percent
,debt,debt,
lem_type,,,
недвижимость,782,10814,0.072314
свадьба,186,2335,0.079657
образование,370,4014,0.092177
автомобиль,403,4308,0.093547


### Вывод

чаще всего просрачиватся кредиты на образование и автомобиль.
соотношение просрочек практичеки одинаково. 

### Шаг 4. Общий вывод

Из наших исследований можно сделать следующий вывод и ответить на вопросы.

**Есть ли зависимость между наличием детей и возвратом кредита в срок?**  
Меньше всего просрочек и бездетных и многодетных.  Прямой зависимости от количества детей и просрочек нет.

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**  
да, зависимость есть. Могу сказать что люди побывавшие в браке просрачивают меньше всех, дальше идут семейные и самые плохие заёмщики одинокие.

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**  
Чаще всего просрачивает средний класс. Богатые и бедные заметно меньше делают просрочки.

**Как разные цели кредита влияют на его возврат в срок?**  
самые просрачиваемые это кредиты на авто и образование.
